In [16]:
# Teemu mac kernel python 3.9.undefined

import datetime as dt
import pandas as pd
import numpy as np
from scipy.stats import norm

In [17]:
def calculate_delta(S, K, T, t, r, sigma, eps):
    d1 = (np.log(S/K) + (r + sigma ** 2 / 2) * (T - t)) / (sigma * np.sqrt(T - t) + eps)
    return norm.cdf(d1)

In [32]:
def calc_vega(S, K, T, t, r, sigma, eps):
    """Calculate option's vega"""
    d1 = (np.log(S/K) + (r + sigma ** 2 / 2) * (T - t)) / (sigma * np.sqrt(T - t) + eps)
    return S * np.sqrt(T-t) * norm.pdf(d1)

In [18]:
# Black-Scholes Call Price
def black_scholes_call(S: float, K: float, T: float, t: float, r: float, sigma: float, eps: float) -> float:
    """
    Calculate Black-Scholes call option price.
    
    Args:
        S: Current stock price
        K: Strike price
        T: Time to maturity
        t: Current time
        r: Risk-free rate
        sigma: Volatility
        eps: Small number for numerical stability
    
    Returns:
        float: Call option price
    """
    d1 = (np.log(S/K) + (r + sigma ** 2 / 2) * (T - t)) / (sigma * np.sqrt(T - t) + eps)
    d2 = d1 - sigma * np.sqrt(T-t)
    return S * norm.cdf(d1) - K * np.exp(-r * (T-t)) * norm.cdf(d2)

In [19]:
def interval_func(start, end, interval):
    result = []
    current = start
    while current <= end:
        result.append(round(current, 4))
        current += interval
    return result

In [20]:
def calc_delta(S, K, T, t, r, C_0, eps=np.finfo(float).eps, tol=0.1):

    for sigma in interval_func(0, 5, 0.001):
        c = black_scholes_call(S=S, K=K, T=T, t=t, r=r, sigma=sigma, eps=eps)
        if abs(c-C_0) <= tol:
            delta = calculate_delta(S=S, K=K, T=T, t=t, r=r, sigma=sigma, eps=eps)
            # print(f"Delta = {delta}, sigma = {sigma}, Price: {S}, S-C={S-C_0} \ncall price from BS = {round(c, 5)}, Call price from data = {C_0}")
            return delta
    # print(f"Delta = 1, sigma = 0, call from BS if sigma = 0: {black_scholes_call(S=S, K=K, T=T, t=t, r=r, sigma=0, eps=eps)} Call price from data = {C_0}")
    return 1

In [ ]:
def calc_delta_recursive(S, K, T, t, r, C_0, eps=np.finfo(float).eps, tol=0.1, max_tol=2.0):
    """
    Recursively calculate delta with increasing tolerance until solution is found
    or max_tolerance is reached
    """
    if tol > max_tol:
        print(f"Warning: No solution found within tolerance {max_tol}")
        return 1
        
    for sigma in interval_func(0, 5, 0.001):
        c = black_scholes_call(S=S, K=K, T=T, t=t, r=r, sigma=sigma, eps=eps)
        if abs(c-C_0) <= tol:
            delta = calculate_delta(S=S, K=K, T=T, t=t, r=r, sigma=sigma, eps=eps)
            return delta
            
    print(f'Increasing tolerance from {tol} to {tol+0.2}')
    return calc_delta_recursive(S, K, T, t, r, C_0, eps, tol+0.2, max_tol)

# Equation (1): Alpha Calculation

$$
\alpha = \frac{\partial C^{BS}}{\partial S} - \frac{\frac{\partial C^{BS}}{\partial \sigma}}{\frac{\partial C^{Rep}}{\partial \sigma}} \cdot \frac{\partial C^{Rep}}{\partial S}
$$

Where:
- $\alpha$ is the hedge ratio for the underlying asset

- $\frac{\partial C^{BS}}{\partial S}$ is the delta of the original Black-Scholes option

- $\frac{\partial C^{BS}}{\partial \sigma}$ is the vega of the original Black-Scholes option

- $\frac{\partial C^{Rep}}{\partial \sigma}$ is the vega of the replicating option

- $\frac{\partial C^{Rep}}{\partial S}$ is the delta of the replicating option


In [ ]:
def calc_alpha(S, K, T, t, r, sigma, eps, T2):
    """Calculate alpha according to equation (1)"""
    # Calculate delta of original option
    delta_BS = calculate_delta(S, K, T, t, r, sigma, eps)
    
    # Calculate vegas
    vega_BS = calc_vega(S, K, T, t, r, sigma, eps)
    vega_rep = calc_vega(S, K, T2, t, r, sigma, eps)
    
    # Calculate delta of replicating option
    delta_rep = calculate_delta(S, K, T2, t, r, sigma, eps)
    
    # Equation (1): α = ∂C^BS/∂S - (∂C^BS/∂σ)/(∂C^Rep/∂σ) * ∂C^Rep/∂S
    alpha = delta_BS - (vega_BS/vega_rep) * delta_rep
    
    return alpha

In [22]:
# Alkuarvot
# Data:
data = pd.read_feather('apple.feather')

In [28]:
# drop unnecessary three rows as our hedging starting day is 2024-09-10
df = data.drop(index=[0,1,2]).reset_index(drop=True)

start_date = dt.datetime.strptime("2024-09-10", '%Y-%m-%d')
maturity_date = dt.datetime.strptime('2024-10-25', '%Y-%m-%d')

# ATM 2024-09-10
T = 45/365
t = 0
K = 220

strike = f'C{K}'
C_0 = float(df[strike].iloc[0])
S_0 = float(df['Underlying'].iloc[0])

# Risk-free rate is us 30-day treasure bond risk-free rate at 2024-09-10
r = 0.0497
eps = np.finfo(float).eps
delta = calc_delta(S=S_0, K=K, T=T, t=t, r=r, C_0=C_0, eps=np.finfo(float).eps, tol=0.1)
print(f"Delta at S_O= {delta}, S_0 = {S_0}, C_0 = {C_0}")

# Hedging interval
interval = 1


Delta at S_O= 0.5473233637793574, S_0 = 220.11, C_0 = 8.65


In [29]:
"""
We want to re-hedge the portfolio at specific intervals and calculate mean square error E = (1 / n - 1) * SUM_i=1->n-1(A^2)
We choose to hedge every second day.
Function calculates both OP and RE portfolio values and their difference A_i as the result.
Re-hedging is done by calculating new delta values.

OP = c_i+1 - c_1
RE = delta_i(s_i+1 - s_i)
A_i = OP + RE
E = (1 / n - 1) * SUM_i=1->n-1(A_i^2)

interval = re-hedging interval (days)
strike = strike price in format ('C{strike_price}) e.g. 'C105'
"""

# init portfolia values
OP_0 = C_0

# delta * S_0
RE_0 = -delta * S_0
print(OP_0, RE_0, OP_0+RE_0)

def hedging(interval, strike, df, delta):
    A_boss = 0
    interval_count = 1
    c_0 = C_0
    c_1 = 0
    s_0 = S_0
    s_1 = 0
    t = 0
    n = 0
    
    for _, row in df.iterrows():
        c_1 = float(row[strike])
        s_1 = float(row['Underlying'])
        t = (45 - (maturity_date - row['Date']).days) / 365
        
        if row.name == 0:
            #print(1)
            continue

        OP = c_1 - c_0

        #print(f'{delta} * ({s_1}-{s_0})')
        RE = -delta * (s_1-s_0)
        #print(f'OP: {OP}, RE: {RE}')
        A = OP + RE
    
        # print(f'Error/Difference OP + RE: {A}')
        A_boss += A ** 2
        n += 1
        
        if interval_count % interval == 0:
          #print(row['Date'].strftime("%Y-%m-%d"))
          delta = calc_delta(s_1, K, T, t, r, c_1)

        c_0 = c_1
        s_0 = s_1
        interval_count += 1

    mse = A_boss/(n-1)
    #print(n)
    return mse


8.65 -120.47134560147437 -111.82134560147436


In [30]:
hedging(interval=interval, strike=strike, df=df, delta=delta)


0.08751974483554929

In [26]:
"""
We want to re-hedge the portfolio at specific intervals and calculate mean square error E = (1 / n - 1) * SUM_i=1->n-1(A^2)
We choose to hedge every second day.
Function calculates both OP and RE portfolio values and their difference A_i as the result.
Re-hedging is done by calculating new delta values.

OP = c_i+1 - c_1
RE = delta_i(s_i+1 - s_i)
A_i = OP + RE
E = (1 / n - 1) * SUM_i=1->n-1(A_i^2)

interval = re-hedging interval (days)
strike = strike price in format ('C{strike_price}) e.g. 'C105'
"""

def hedging_loopped(interval, strike, df, delta, C_0, S_0, maturity_date, T, r, RE_0, OP_0):
    A_boss = 0
    interval_count = 1
    c_0 = C_0
    c_1 = 0
    s_0 = S_0
    s_1 = 0
    t = 0
    n = 0

    K = float(strike.split('C')[1])

    for _, row in df.iterrows():
        c_1 = float(row[strike])
        s_1 = float(row['Underlying'])
        t = (45 - (maturity_date - row['Date']).days) / 365
    
        if row.name == 0:
            continue

        OP = c_1 - c_0

        #print(f'{delta} * ({s_1}-{s_0})')
        RE = -delta * (s_1-s_0)
        # print(f'OP: {OP}, RE: {RE}')
        A = OP + RE
        #print(f'Error/Difference OP + RE: {A}')
        A_boss += A ** 2
        n += 1
        
        if interval_count % interval == 0:
          #print(row['Date'].strftime("%Y-%m-%d"))
          delta = calc_delta(s_1, K, T, t, r, c_1)
          

        c_0 = c_1
        s_0 = s_1
        interval_count += 1

    mse = A_boss/(n-1)
    
    return mse



In [31]:
# Alkuarvot
# Data:
data = pd.read_feather('apple25102024.feather')

# drop unnecessary three rows as our hedging starting day is 2024-09-10
df = data.drop(index=[0,1,2]).reset_index(drop=True)

start_date = dt.datetime.strptime("2024-09-10", '%Y-%m-%d')
maturity_date = dt.datetime.strptime('2024-10-25', '%Y-%m-%d')
strikes = [215, 220, 225, 230, 235, 240, 245, 250]
results = []
for k in strikes:
    print(f"-------------- strike: {k} ----------------")
    #ATM 2024-09-10
    T = 45/365
    t = 0
    K = k

    strike = f'C{K}'
    C_0 = float(df[strike].iloc[0])
    S_0 = float(df['Underlying'].iloc[0])

    # Risk-free rate is us 30-day treasure bond risk-free rate at 2024-09-10
    r = 0.0497
    eps = np.finfo(float).eps
    delta = calc_delta_recursive(S=S_0, K=K, T=T, t=t, r=r, C_0=C_0, eps=np.finfo(float).eps, tol=0.1)
    # print(f"Delta at S_O= {delta}, S_0 = {S_0}, C_0 = {C_0}")

    # Hedging interval
    interval = 1
    mse_s = []
    # init portfolia values
    OP_0 = C_0

    # delta * S_0
    RE_0 = delta * S_0
    
    error = hedging_loopped(interval, strike, df, delta, C_0, S_0, maturity_date, T, r, RE_0, OP_0)
    results.append({"Strike": K, "MSE": error})
    print(f"MSE for strike {K} is {error}")
    

results_df = pd.DataFrame(results)

-------------- strike: 215 ----------------
MSE for strike 215 is 0.40608087461280457
-------------- strike: 220 ----------------
MSE for strike 220 is 0.08751974483554929
-------------- strike: 225 ----------------
MSE for strike 225 is 0.12628700973920648
-------------- strike: 230 ----------------
MSE for strike 230 is 0.10257345425550289
-------------- strike: 235 ----------------
MSE for strike 235 is 0.12009453018789852
-------------- strike: 240 ----------------
MSE for strike 240 is 0.051460789084500136
-------------- strike: 245 ----------------
MSE for strike 245 is 0.023172443873171005
-------------- strike: 250 ----------------
MSE for strike 250 is 0.0062997646322188105


In [ ]:
# Usage example:
T = 45/365  # Original option maturity
T2 = 90/365  # Longer maturity for replicating option

def delta_vega_hedging(interval, strike, df, delta, C_0, S_0, maturity_date, T, T2, r, RE_0, OP_0):
    """
    Delta-vega hedging implementation using the theoretical formulas
    
    Args:
        T: maturity of original option
        T2: maturity of replicating option (longer)
        
    The replicating portfolio RE consists of:
    - α amount of underlying stock
    - η amount of replicating option (longer maturity)
    """
    A_boss = 0
    interval_count = 1
    c_0 = C_0
    c_1 = 0
    s_0 = S_0
    s_1 = 0
    t = 0
    n = 0
    
    K = float(strike.split('C')[1])
    rep_strike = f'C{K}_T2'  # Longer maturity option
    
    # Initial calculations
    sigma = calc_implied_vol(S_0, K, T, 0, r, c_0)
    
    # Calculate initial positions
    vega_BS = calc_vega(S_0, K, T, 0, r, sigma, eps)
    vega_rep = calc_vega(S_0, K, T2, 0, r, sigma, eps)
    eta = -vega_BS / vega_rep  # Amount of replicating option
    alpha = calc_alpha(S_0, K, T, 0, r, sigma, eps, T2)  # Amount of underlying
    
    for _, row in df.iterrows():
        c_1 = float(row[strike])  # Original option price
        c_rep = float(row[rep_strike])  # Replicating option price
        s_1 = float(row['Underlying'])
        
        # Calculate time to maturity for both options
        t_original = (T - (maturity_date - row['Date']).days) / 365
        t_replicating = (T2 - (maturity_date - row['Date']).days) / 365
        
        if row.name == 0:
            continue
            
        # Calculate portfolio changes
        OP = c_1 - c_0  # Change in original option
        
        # Calculate replicating portfolio change
        stock_change = -alpha * (s_1 - s_0)  # Change from stock position
        rep_option_change = -eta * (c_rep - c_rep_prev)  # Change from replicating option
        RE = stock_change + rep_option_change
        
        # Calculate error
        A = OP + RE
        A_boss += A ** 2
        n += 1
        
        if interval_count % interval == 0:
            # Recalculate positions
            sigma = calc_implied_vol(s_1, K, T, t_original, r, c_1)
            
            # Update hedge ratios
            vega_BS = calc_vega(s_1, K, T, t_original, r, sigma, eps)
            vega_rep = calc_vega(s_1, K, T2, t_replicating, r, sigma, eps)
            eta = -vega_BS / vega_rep
            alpha = calc_alpha(s_1, K, T, t_original, r, sigma, eps, T2)
            
        # Store current values for next iteration
        c_0 = c_1
        c_rep_prev = c_rep
        s_0 = s_1
        interval_count += 1
        
    mse = A_boss/(n-1)
    return mse


In [ ]:
# Usage example with both hedging strategies
results = []
for k in strikes:
    print(f"-------------- strike: {k} ----------------")
    strike = f'C{k}'
    C_0 = float(df[strike].iloc[0])
    S_0 = float(df['Underlying'].iloc[0])
    
    # Delta hedging
    delta = calc_delta_recursive(S=S_0, K=k, T=T, t=t, r=r, C_0=C_0)
    delta_error = hedging_loopped(interval, strike, df, delta, C_0, S_0, 
                                maturity_date, T, r, -delta * S_0, C_0)
    
    # Delta-vega hedging
    sigma = calc_implied_vol(S_0, k, T, 0, r, C_0)
    vega_BS = calc_vega(S_0, k, T, 0, r, sigma, eps)
    vega_rep = calc_vega(S_0, k, T2, 0, r, sigma, eps)
    eta = -vega_BS / vega_rep
    alpha = calc_alpha(S_0, k, T, 0, r, sigma, eps, T2)
    
    delta_vega_error = delta_vega_hedging(interval, strike, df, delta, C_0, S_0, 
                                        maturity_date, T, T2, r, 
                                        -(alpha * S_0 + eta * C_0_rep), C_0)
    
    results.append({
        "Strike": k,
        "Delta_MSE": delta_error,
        "Delta_Vega_MSE": delta_vega_error
    })
    print(f"Strike {k}:")
    print(f"Delta MSE = {delta_error:.6f}")
    print(f"Delta-Vega MSE = {delta_vega_error:.6f}")

# Create comparison DataFrame
results_df = pd.DataFrame(results)
print("\nComparison of Hedging Strategies:")
print(results_df)


In [ ]:
# Optional: Create visualization
import matplotlib.pyplot as plt

plt.figure(figsize=(10, 6))
plt.plot(results_df['Strike'], results_df['Delta_MSE'], label='Delta Hedging', marker='o')
plt.plot(results_df['Strike'], results_df['Delta_Vega_MSE'], label='Delta-Vega Hedging', marker='o')
plt.xlabel('Strike Price')
plt.ylabel('Mean Squared Error')
plt.title('Comparison of Hedging Strategies')
plt.legend()
plt.grid(True)
plt.show()